# 03 — Entrenamiento y Evaluación (SUPERVISED y CONTINUAL con EWC/NAIVE)

Este notebook entrena un modelo **SNN** para **regresión del ángulo de dirección (steering)** en dos protocolos:

- **Supervised** sobre `circuito1`.
- **Continual** con dos tareas secuenciales `circuito1 → circuito2` usando:
  - **EWC** (consolidación elástica de pesos), o
  - **NAIVE** (baseline sin penalización; equivalente a λ=0).

> **Requisitos previos**: Ejecuta `01_DATA_QC_PREP.ipynb` para generar `train/val/test.csv` y `tasks.json`.


In [1]:
# =============================================================================
# Imports y setup
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

#from src.utils import set_seeds, load_preset
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model, default_tfm_for_model
from src.training import TrainConfig
#from src.eval import eval_loader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

MODEL_NAME = "pilotnet_snn"   # "snn_vision" | "pilotnet_ann" | "pilotnet_snn"

# tfm por defecto según el modelo:
tfm = default_tfm_for_model(MODEL_NAME, to_gray=True)

def make_model_fn(tfm):
    # kwargs solo necesarios para pilotnet_snn; ignorados por otros
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)

torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


In [2]:
GPU_ENCODE = True

SAFE_MODE = False
NUM_WORKERS    = 12
PREFETCH       = 2
PIN_MEMORY     = True
PERSISTENT     = True

AUG_CFG_LIGHT = AugmentConfig(prob_hflip=0.5, brightness=None, gamma=None, noise_std=0.0)
AUG_CFG_FULL  = AugmentConfig(prob_hflip=0.5, brightness=(0.9, 1.1), gamma=(0.95, 1.05), noise_std=0.005)
AUG_CFG = AUG_CFG_LIGHT

USE_OFFLINE_BALANCED = True
USE_ONLINE_BALANCING = False

if SAFE_MODE:
    NUM_WORKERS = 0
    PREFETCH = None
    PIN_MEMORY = False
    PERSISTENT = False
    USE_OFFLINE_BALANCED = False
    USE_ONLINE_BALANCING = False
    AUG_CFG = None

print(f"[SAFE_MODE={SAFE_MODE}] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")


[SAFE_MODE=False] workers=12 prefetch=2 pin=True persistent=True


In [3]:
# =============================================================================
# Verificación de datos (normal y, si existe, balanceado offline)
# =============================================================================
from pathlib import Path

RAW  = ROOT/"data"/"raw"/"udacity"
PROC = ROOT/"data"/"processed"

RUNS = ["circuito1","circuito2"]  # ajusta si hace falta

missing = []
for run in RUNS:
    base = PROC / run

    # Comprobación obligatoria: splits normales
    for part in ["train","val","test"]:
        p = base / f"{part}.csv"
        if not p.exists():
            missing.append(str(p))

    # Comprobación opcional: train_balanced.csv (para modo OFFLINE balanceado)
    p_bal = base / "train_balanced.csv"
    if p_bal.exists():
        print(f"✓ {p_bal} OK")
    else:
        print(f"⚠️  Falta {p_bal}. Si más abajo pones USE_OFFLINE_BALANCED=True, "
              f"ejecuta 01A_PREP_BALANCED.ipynb o el script tools/make_splits_balanced.py")

if missing:
    raise FileNotFoundError(
        "Faltan CSV obligatorios (ejecuta 01A_PREP_BALANCED.ipynb o tu pipeline de prep):\n"
        + "\n".join(" - " + m for m in missing)
    )

print("OK: splits 'train/val/test' encontrados.")


✓ /home/cesar/proyectos/TFM_SNN/data/processed/circuito1/train_balanced.csv OK
✓ /home/cesar/proyectos/TFM_SNN/data/processed/circuito2/train_balanced.csv OK
OK: splits 'train/val/test' encontrados.


In [4]:
# ===================== Balanceo: helper =====================
print(
    "Modo balanceo:",
    "OFFLINE (tasks_balanced.json)" if USE_OFFLINE_BALANCED else "ORIGINAL (tasks.json)",
    "| Balanceo ONLINE:", USE_ONLINE_BALANCING
)

# Seguridad anti doble balanceo:
if USE_OFFLINE_BALANCED and USE_ONLINE_BALANCING:
    raise RuntimeError("Doble balanceo detectado: OFFLINE y ONLINE a la vez. "
                       "Pon USE_ONLINE_BALANCING=False cuando uses train_balanced.csv.")

from pathlib import Path  # (omite esta línea si ya importaste Path arriba)

def _balance_flag(train_csv_path: str | Path) -> bool:
    """
    Activa balanceo ONLINE solo si:
    - USE_ONLINE_BALANCING == True
    - Y el CSV de train NO es 'train_balanced.csv'
    """
    is_balanced_csv = Path(train_csv_path).name == "train_balanced.csv"
    return bool(USE_ONLINE_BALANCING and not is_balanced_csv)


Modo balanceo: OFFLINE (tasks_balanced.json) | Balanceo ONLINE: False


In [5]:
import src.training as training

In [6]:
# =============================================================================
# Elegir split: normal (tasks.json) o balanceado offline (tasks_balanced.json)
# =============================================================================
with open(PROC / ("tasks_balanced.json" if USE_OFFLINE_BALANCED else "tasks.json"), "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

# Vista rápida: muestra el CSV de train que se usará por cada tarea
print("Tareas y su TRAIN CSV:")
for t in task_list:
    print(f" - {t['name']}: {Path(t['paths']['train']).name}")

task_list[:2]  # vista rápida

# Guardarraíl extra: si has activado el OFFLINE balanceado,
# exige que el 'train' sea train_balanced.csv y que exista.
if USE_OFFLINE_BALANCED:
    for t in task_list:
        train_path = Path(t["paths"]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' pero encontré '{train_path.name}'. "
                "Repite 01A_PREP_BALANCED.ipynb o ajusta USE_OFFLINE_BALANCED=False."
            )
        if not train_path.exists():
            raise FileNotFoundError(
                f"[{t['name']}] No existe {train_path}. Genera los balanceados con 01A_PREP_BALANCED.ipynb."
            )
    print("✔ Verificación OFFLINE balanceado superada (train_balanced.csv por tarea).")


Tareas y su TRAIN CSV:
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
✔ Verificación OFFLINE balanceado superada (train_balanced.csv por tarea).


In [7]:
from pathlib import Path
from src.utils import make_loaders_from_csvs

def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed, **dl_kwargs):
    RAW = ROOT / "data" / "raw" / "udacity" / task["name"]
    paths = task["paths"]

    # Si vamos a codificar en GPU, pedimos 4D (image) al loader;
    # si no, dejamos el encoder temporal en el propio dataset.
    encoder_for_loader = "image" if (GPU_ENCODE and encoder in {"rate","latency","raw"}) else encoder

    return make_loaders_from_csvs(
        base_dir=RAW,
        train_csv=Path(paths["train"]),
        val_csv=Path(paths["val"]),
        test_csv=Path(paths["test"]),
        batch_size=batch_size,
        encoder=encoder_for_loader,
        T=T, gain=gain, tfm=tfm, seed=SEED,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        persistent_workers=PERSISTENT,
        prefetch_factor=PREFETCH,
        # online balancing opcional:
        aug_train=AUG_CFG,
        balance_train=(USE_ONLINE_BALANCING and Path(paths["train"]).name != "train_balanced.csv"),
        balance_bins=21,
        balance_smooth_eps=1e-3,
    )


In [8]:
from src.runner import run_continual

In [9]:
# === Activar métrica de it/s por época (parche temporal) ===
import time, json
from pathlib import Path
import torch
from torch import nn, optim
from torch.amp import autocast, GradScaler
import src.training as training
from src.utils import set_seeds  # ya lo tienes importado en el notebook

# Guarda la referencia al original para poder restaurar luego
orig_train_supervised = training.train_supervised

def train_supervised_ips(model: nn.Module, train_loader, val_loader, loss_fn: nn.Module,
                         cfg, out_dir: Path, method=None):
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    if cfg.seed is not None:
        set_seeds(cfg.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    opt = optim.Adam(model.parameters(), lr=cfg.lr)

    use_amp = bool(cfg.amp and torch.cuda.is_available())
    scaler = GradScaler(enabled=use_amp)

    history = {"train_loss": [], "val_loss": []}
    t0_total = time.perf_counter()

    for epoch in range(1, cfg.epochs + 1):
        model.train()
        running = 0.0
        nb = 0
        t_epoch0 = time.perf_counter()

        for x, y in train_loader:
            # encode/permutación runtime y subida a device (usa tu helper actual)
            x = training._permute_if_needed(x).to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            opt.zero_grad(set_to_none=True)
            with autocast("cuda", enabled=use_amp):
                y_hat = model(x)
                loss = loss_fn(y_hat, y)
                if method is not None:
                    loss = loss + method.penalty()

            if use_amp:
                scaler.scale(loss).backward()
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(opt); scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                opt.step()

            running += loss.item()
            nb += 1

        epoch_time = time.perf_counter() - t_epoch0
        ips = nb / epoch_time if epoch_time > 0 else float("nan")
        print(f"[TRAIN it/s] epoch {epoch}/{cfg.epochs}: {ips:.1f} it/s  "
              f"({nb} iters en {epoch_time:.2f}s)")

        train_loss = running / max(1, nb)

        # --- validación ---
        model.eval()
        v_running = 0.0; nvb = 0
        with torch.no_grad():
            for x, y in val_loader:
                x = training._permute_if_needed(x).to(device, non_blocking=True)
                y = y.to(device, non_blocking=True)
                with autocast("cuda", enabled=use_amp):
                    y_hat = model(x)
                    v_loss = loss_fn(y_hat, y)
                v_running += v_loss.item(); nvb += 1
        val_loss = v_running / max(1, nvb)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

    elapsed = time.perf_counter() - t0_total
    manifest = {
        "epochs": cfg.epochs, "batch_size": cfg.batch_size, "lr": cfg.lr,
        "amp": cfg.amp, "seed": cfg.seed, "elapsed_sec": elapsed,
        "device": str(device), "history": history,
    }
    (out_dir / "manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    return history

# Activa el parche
training.train_supervised = train_supervised_ips
print("✅ it/s por época ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised")

✅ it/s por época ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised


In [10]:
# ====================== CONFIG PARA COMPARATIVAS ======================

# Qué métodos lanzar (puedes comentar/descomentar líneas)
METHOD_SPECS = {
    "naive": [
        {},  # sin hiperparámetros
    ],
    "ewc": [
        {"lam": 7e8, "fisher_batches": 1000},
        {"lam": 1e9, "fisher_batches": 1000},
    ],
    "rehearsal": [
        {"buffer_size": 5000, "replay_ratio": 0.2},
    ],
    "rehearsal+ewc": [
        {"buffer_size": 5000, "replay_ratio": 0.2, "lam": 7e8, "fisher_batches": 1000},
        {"buffer_size": 5000, "replay_ratio": 0.2, "lam": 1e9, "fisher_batches": 1000},
    ],
}

# Semillas / encoders / presets
# SEEDS      = [42, 43, 44]     # o [42]
SEEDS      = [42]     # o [42]
ENCODERS   = ["rate"]         # puedes añadir "latency"
# PRESETS_TO_RUN = ["fast", "std", "accurate"]
PRESETS_TO_RUN = ["fast"]

# Fisher por preset (para EWC)
FISHER_BY_PRESET = {"fast": 800, "std": 1000, "accurate": 1500}

print("Métodos:", METHOD_SPECS)
print("Presets:", PRESETS_TO_RUN, "| Seeds:", SEEDS, "| Encoders:", ENCODERS)
print("Fisher/batches:", FISHER_BY_PRESET)


Métodos: {'naive': [{}], 'ewc': [{'lam': 700000000.0, 'fisher_batches': 1000}, {'lam': 1000000000.0, 'fisher_batches': 1000}], 'rehearsal': [{'buffer_size': 5000, 'replay_ratio': 0.2}], 'rehearsal+ewc': [{'buffer_size': 5000, 'replay_ratio': 0.2, 'lam': 700000000.0, 'fisher_batches': 1000}, {'buffer_size': 5000, 'replay_ratio': 0.2, 'lam': 1000000000.0, 'fisher_batches': 1000}]}
Presets: ['fast'] | Seeds: [42] | Encoders: ['rate']
Fisher/batches: {'fast': 800, 'std': 1000, 'accurate': 1500}


In [11]:
# ====================== DRIVER MULTISEMILLAS ======================
for enc in ENCODERS:
    for seed in SEEDS:
        for preset_i in PRESETS_TO_RUN:
            for method, specs in METHOD_SPECS.items():
                for spec_ in specs:
                    print(f"\n=== RUN: preset={preset_i} | method={method} | seed={seed} | enc={enc} | kwargs={spec_} ===")
                    out_path, _ = run_continual(
                        task_list=task_list,
                        make_loader_fn=make_loader_fn,
                        make_model_fn=make_model_fn,
                        tfm=tfm,
                        preset=preset_i,
                        method=method,
                        seed=seed,
                        encoder=enc,
                        runtime_encode=GPU_ENCODE,
                        out_root=ROOT/"outputs",
                        verbose=True,
                        method_kwargs=spec_,   # <- único sitio
                    )
                    print("OK:", out_path)


print("\n Listo. Ejecuta las celdas de resumen.")



=== RUN: preset=fast | method=naive | seed=42 | enc=rate | kwargs={} ===

--- Tarea 1/2: circuito1 | preset=fast | method=naive | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=naive | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_naive_rate_model-PilotNetSNN_66x200_gray_seed_42

=== RUN: preset=fast | method=ewc | seed=42 | enc=rate | kwargs={'lam': 700000000.0, 'fisher_batches': 1000} ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_7e+08 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=ewc_lam_7e+08 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42

=== RUN: preset=fast | method=ewc | seed=42 | enc=rate | kwargs={'lam': 1000000000.0, 'fisher_batches': 1000} ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_1e+09 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


...

  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_rehearsal_buf_5000_rr_20_rate_model-PilotNetSNN_66x200_gray_seed_42

=== RUN: preset=fast | method=rehearsal+ewc | seed=42 | enc=rate | kwargs={'buffer_size': 5000, 'replay_ratio': 0.2, 'lam': 700000000.0, 'fisher_batches': 1000} ===

--- Tarea 1/2: circuito1 | preset=fast | method=rehearsal_buf_5000_rr_20+ewc_lam_7e+08 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=rehearsal_buf_5000_rr_20+ewc_lam_7e+08 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_rehearsal_buf_5000_rr_20+ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42

=== RUN: preset=fast | method=rehearsal+ewc | seed=42 | enc=rate | kwargs={'buffer_size': 5000, 'replay_ratio': 0.2, 'lam': 1000000000.0, 'fisher_batches': 1000} ===

--- Tarea 1/2: circuito1 | preset=fast | method=rehearsal_buf_5000_rr_20+ewc_lam_1e+09 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=rehearsal_buf_5000_rr_20+ewc_lam_1e+09 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_rehearsal_buf_5000_rr_20+ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42

 Listo. Ejecuta las celdas de resumen.


In [12]:
# ¿Qué runs ve y cuáles tienen continual_results.json?
from pathlib import Path
cand = sorted((ROOT/"outputs").glob("continual_*"))
print("Encontrados:", len(cand))
for p in cand:
    print(" -", p.name, "| results.json:", (p/"continual_results.json").exists())


Encontrados: 6
 - continual_fast_ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_naive_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_rehearsal_buf_5000_rr_20+ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_rehearsal_buf_5000_rr_20+ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_rehearsal_buf_5000_rr_20_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True


In [13]:
# =============================================================================
# Resumen comparativo de todos los continual_* en outputs/
# =============================================================================
import re, json
from pathlib import Path
import pandas as pd
from IPython.display import display

ALLOWED_ENC = r"(rate|latency|raw|image)"

def parse_exp_name(name: str):
    """
    Formato esperado (compatible hacia atrás):
      continual_<preset>_<tag>_<encoder>[_model-<model>]?_seed_<seed>?
    donde <tag> puede ser:
      naive
      ewc_lam_1e+09
      rehearsal
      rehearsal+ewc_lam_1e+09
    y <model> es algo tipo: PilotNetSNN_66x200_gray
    """
    pat = re.compile(
        rf"^continual_"
        rf"(?P<preset>[^_]+)_"                # preset
        rf"(?P<tag>.+)_"                      # tag (e.g., ewc_lam_1e+09, rehearsal+ewc_lam_1e+09)
        rf"(?P<enc>{ALLOWED_ENC})"            # encoder
        rf"(?:_model\-(?P<model>.+?))?"       # NUEVO: _model-... (opcional, no codicioso)
        rf"(?:_seed_(?P<seed>\d+))?$"         # seed (opcional)
    )
    m = pat.match(name)
    meta = {"preset": None, "method": None, "lambda": None, "encoder": None, "seed": None, "model": None}
    if not m:
        return meta

    preset = m.group("preset")
    tag    = m.group("tag")
    enc    = m.group("enc")
    seed   = m.group("seed")
    model  = m.group("model")  # puede ser None para runs antiguos

    # λ si está presente dentro del tag
    lam = None
    mlam = re.search(r"_lam_([^_]+)", tag)
    if mlam:
        lam = mlam.group(1)
        method = tag.replace(f"_lam_{lam}", "")
    else:
        method = tag

    return {
        "preset":  preset,
        "method":  method,           # p.ej. "ewc", "naive", "rehearsal+ewc", "rehearsal"
        "lambda":  lam,              # p.ej. "1e+09" o None
        "encoder": enc,              # "rate" | "latency" | "raw" | "image"
        "seed":    int(seed) if seed is not None else None,
        "model":   model,            # e.g., "PilotNetSNN_66x200_gray" (o None)
    }

rows = []
root_out = ROOT / "outputs"

for exp_dir in sorted(root_out.glob("continual_*")):
    name = exp_dir.name
    meta = parse_exp_name(name)
    if meta["preset"] is None:
        continue

    results_path = exp_dir / "continual_results.json"
    if not results_path.exists():
        continue

    with open(results_path, "r", encoding="utf-8") as f:
        res = json.load(f)

    task_names = list(res.keys())
    if len(task_names) < 2:
        # si solo hubo 1 tarea, no hay 'after_*' y no podemos calcular olvido
        continue

    def is_last(d: dict) -> bool:
        return not any(k.startswith("after_") for k in d.keys())

    last_task = None
    first_task = None
    for tn in task_names:
        if is_last(res[tn]):
            last_task = tn
        else:
            first_task = tn

    if first_task is None or last_task is None:
        task_names_sorted = sorted(task_names)
        first_task = task_names_sorted[0]
        last_task  = task_names_sorted[-1]

    c1, c2 = first_task, last_task
    c1_test_mae     = float(res[c1].get("test_mae", float("nan")))
    c2_test_mae     = float(res[c2].get("test_mae", float("nan")))
    after_key_mae   = f"after_{c2}_mae"
    c1_after_c2_mae = float(res[c1].get(after_key_mae, float("nan")))
    forgetting_abs  = c1_after_c2_mae - c1_test_mae
    forgetting_rel  = (forgetting_abs / c1_test_mae * 100.0) if c1_test_mae == c1_test_mae else float("nan")

    rows.append({
        "exp": name,
        "preset":  meta["preset"],
        "method":  meta["method"],
        "lambda":  meta["lambda"],
        "encoder": meta["encoder"],
        "model":   meta["model"],         # NUEVO
        "seed":    meta["seed"],
        "c1_name": c1,
        "c2_name": c2,
        "c1_mae":  c1_test_mae,
        "c1_after_c2_mae": c1_after_c2_mae,
        "c1_forgetting_mae_abs": forgetting_abs,
        "c1_forgetting_mae_rel_%": forgetting_rel,
        "c2_mae":  c2_test_mae,
    })

df = pd.DataFrame(rows)
print(f"runs en resumen: {len(df)}")

if not df.empty:
    df["lambda_num"] = pd.to_numeric(df["lambda"], errors="coerce")
    df["seed"] = pd.to_numeric(df["seed"], errors="coerce").astype("Int64")
    # Ordena incluyendo 'model' (los runs viejos tendrán model NaN y quedarán al final de su grupo)
    df = df.sort_values(
        by=["preset", "method", "encoder", "model", "lambda_num", "seed"],
        na_position="last",
        ignore_index=True,
    )
    display(df[[
        "exp","preset","method","lambda","encoder","model","seed",
        "c1_name","c2_name","c1_mae","c1_after_c2_mae",
        "c1_forgetting_mae_abs","c1_forgetting_mae_rel_%","c2_mae","lambda_num"
    ]])
else:
    print("No hay filas (¿no existen JSONs o sólo hubo 1 tarea por run?).")


runs en resumen: 6


,exp,preset,method,lambda,encoder,model,seed,c1_name,c2_name,c1_mae,c1_after_c2_mae,c1_forgetting_mae_abs,c1_forgetting_mae_rel_%,c2_mae,lambda_num
0,continual_fast_ewc_lam_7e+08_lam_7e+08_rate_mo...,fast,ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.166709,0.164899,-0.001810,-1.085560,0.248751,7.000000e+08
1,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.166709,0.161644,-0.005064,-3.037920,0.231523,1.000000e+09
2,continual_fast_naive_rate_model-PilotNetSNN_66...,fast,naive,None,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.167532,0.271834,0.104302,62.258099,0.165119,NaN
3,continual_fast_rehearsal_buf_5000_rr_20_rate_m...,fast,rehearsal_buf_5000_rr_20,None,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.167532,0.144561,-0.022971,-13.711262,0.182657,NaN
4,continual_fast_rehearsal_buf_5000_rr_20+ewc_la...,fast,rehearsal_buf_5000_rr_20+ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.166709,0.166750,0.000041,0.024673,0.270312,7.000000e+08
5,continual_fast_rehearsal_buf_5000_rr_20+ewc_la...,fast,rehearsal_buf_5000_rr_20+ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.166709,0.164600,-0.002109,-1.265038,0.242203,1.000000e+09


In [14]:
import pandas as pd
from IPython.display import display

if df.empty:
    print("df está vacío; salta agregados.")
else:
    cols_metrics = ["c1_mae", "c1_after_c2_mae", "c1_forgetting_mae_abs", "c1_forgetting_mae_rel_%", "c2_mae"]
    gdf = df.copy()
    if "lambda_num" not in gdf.columns:
        gdf["lambda_num"] = pd.to_numeric(gdf["lambda"], errors="coerce")

    agg = (
        gdf
        .groupby(["preset", "method", "encoder", "lambda", "lambda_num"], dropna=False)[cols_metrics]
        .agg(["mean", "std", "count"])
        .reset_index()
    )
    agg.columns = ["_".join(filter(None, map(str, col))).rstrip("_") for col in agg.columns.to_flat_index()]
    agg = agg.sort_values(by=["preset", "method", "encoder", "lambda_num"], na_position="last", ignore_index=True)

    summary_dir = ROOT / "outputs" / "summary"
    summary_dir.mkdir(parents=True, exist_ok=True)
    agg.to_csv(summary_dir / "continual_summary_agg.csv", index=False)
    print("Guardado:", summary_dir / "continual_summary_agg.csv")
    display(agg.head(20))


Guardado: /home/cesar/proyectos/TFM_SNN/outputs/summary/continual_summary_agg.csv


,preset,method,encoder,lambda,lambda_num,c1_mae_mean,c1_mae_std,c1_mae_count,c1_after_c2_mae_mean,c1_after_c2_mae_std,c1_after_c2_mae_count,c1_forgetting_mae_abs_mean,c1_forgetting_mae_abs_std,c1_forgetting_mae_abs_count,c1_forgetting_mae_rel_%_mean,c1_forgetting_mae_rel_%_std,c1_forgetting_mae_rel_%_count,c2_mae_mean,c2_mae_std,c2_mae_count
0,fast,ewc,rate,7e+08,7.000000e+08,0.166709,NaN,1,0.164899,NaN,1,-0.001810,NaN,1,-1.085560,NaN,1,0.248751,NaN,1
1,fast,ewc,rate,1e+09,1.000000e+09,0.166709,NaN,1,0.161644,NaN,1,-0.005064,NaN,1,-3.037920,NaN,1,0.231523,NaN,1
2,fast,naive,rate,NaN,NaN,0.167532,NaN,1,0.271834,NaN,1,0.104302,NaN,1,62.258099,NaN,1,0.165119,NaN,1
3,fast,rehearsal_buf_5000_rr_20,rate,NaN,NaN,0.167532,NaN,1,0.144561,NaN,1,-0.022971,NaN,1,-13.711262,NaN,1,0.182657,NaN,1
4,fast,rehearsal_buf_5000_rr_20+ewc,rate,7e+08,7.000000e+08,0.166709,NaN,1,0.166750,NaN,1,0.000041,NaN,1,0.024673,NaN,1,0.270312,NaN,1
5,fast,rehearsal_buf_5000_rr_20+ewc,rate,1e+09,1.000000e+09,0.166709,NaN,1,0.164600,NaN,1,-0.002109,NaN,1,-1.265038,NaN,1,0.242203,NaN,1


In [15]:
from IPython.display import display

if df.empty or 'agg' not in globals():
    print("No hay datos agregados; salta tabla formateada.")
else:
    def fmt(x, prec=4):
        import pandas as pd
        return "" if pd.isna(x) else f"{x:.{prec}f}"

    show = agg.copy()
    count_cols = [c for c in show.columns if c.endswith("_count")]
    if count_cols:
        show["count"] = show[count_cols[0]].astype("Int64")
        show = show.drop(columns=count_cols)

    for c in [c for c in show.columns if c.endswith("_mean") or c.endswith("_std")]:
        show[c] = show[c].map(lambda v: fmt(v, 4))

    cols = [
        "preset", "method", "encoder", "lambda",
        "c1_mae_mean", "c1_forgetting_mae_rel_%_mean", "c2_mae_mean",
        "c1_mae_std",  "c1_forgetting_mae_rel_%_std",  "c2_mae_std",
        "count"
    ]
    missing = [c for c in cols if c not in show.columns]
    if missing:
        print("Aviso: faltan columnas en 'show':", missing)
        cols = [c for c in cols if c in show.columns]

    show = show[cols].rename(columns={
        "preset": "preset",
        "method": "método",
        "encoder": "codificador",
        "lambda": "λ",
        "c1_mae_mean": "MAE Tarea1 (media)",
        "c1_forgetting_mae_rel_%_mean": "Olvido T1 (%) (media)",
        "c2_mae_mean": "MAE Tarea2 (media)",
        "c1_mae_std": "MAE Tarea1 (σ)",
        "c1_forgetting_mae_rel_%_std": "Olvido T1 (%) (σ)",
        "c2_mae_std": "MAE Tarea2 (σ)",
        "count": "n (semillas)"
    })
    display(show)


,preset,método,codificador,λ,MAE Tarea1 (media),Olvido T1 (%) (media),MAE Tarea2 (media),MAE Tarea1 (σ),Olvido T1 (%) (σ),MAE Tarea2 (σ),n (semillas)
0,fast,ewc,rate,7e+08,0.1667,-1.0856,0.2488,,,,1
1,fast,ewc,rate,1e+09,0.1667,-3.0379,0.2315,,,,1
2,fast,naive,rate,NaN,0.1675,62.2581,0.1651,,,,1
3,fast,rehearsal_buf_5000_rr_20,rate,NaN,0.1675,-13.7113,0.1827,,,,1
4,fast,rehearsal_buf_5000_rr_20+ewc,rate,7e+08,0.1667,0.0247,0.2703,,,,1
5,fast,rehearsal_buf_5000_rr_20+ewc,rate,1e+09,0.1667,-1.2650,0.2422,,,,1
